# Import Libraries

In [3]:
pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4


In [4]:
!pip install --no-cache-dir transformers sentencepiece datasets torchsummaryX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 248.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 171.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 128.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 kB 204.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 148.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 148.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 125.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 126.4 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [5]:
from datasets import load_dataset
from transformers import BertTokenizer, BertModel, AutoTokenizer
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
import gc
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from collections import Counter
from torchsummaryX import summary
from sklearn.metrics import accuracy_score
import re
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cpu


In [6]:
#将当前位置锚定在google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preprocessing

In [7]:
#paragraph encoder
#初始化tokenizer（使得一句话分解成token并返回token_ID）
tokenizer1 = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny", do_lower_case=True)
#初始化encoder（将token_ID根据上下文转化为向量）
encoder1 = BertModel.from_pretrained("prajjwal1/bert-tiny")

#一个演示例子
i = tokenizer1('This is a tokenization example, I love swimming', return_tensors="pt")
o = encoder1(**i, output_hidden_states=True)
embedding = o.last_hidden_state.detach().numpy()
#返回了12个token，每个token是一个长度为128的向量来表达其意
print(embedding.shape)

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(1, 12, 128)


In [8]:
#download training data
datasets = load_dataset("squad_v2")
datasets

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [9]:
#这个函数按一定比例抽取上下文context中的答案和非答案部分，返回原文相应短语的embedding，和这个短语是答案（1）还是非答案（0）
def extract_and_tokenize(context, answers, tokenizer, encoder):
    embeddings, tokenized_context, _ = tokenize_and_encode_context(context, tokenizer, encoder)
    binary_labels = generate_binary_labels(answers, tokenized_context, tokenizer)
    embeddings, binary_labels = extract_answer_token_and_nonanswer_token(embeddings, binary_labels)
    return embeddings, binary_labels

In [10]:
def tokenize_and_encode_context(context, tokenizer, encoder):
    tokenized_context = []
    embeddings = []
    for text in context:
        tokenization = tokenizer(text, return_tensors="pt", max_length = 512, 
                                 truncation = True, padding='max_length')
        context_token = tokenization['input_ids'][0].detach().numpy()
        tokenized_context.append(context_token)
        out = encoder(**tokenization, output_hidden_states=True)
        embedding = out.last_hidden_state.detach().numpy()
        assert(embedding.shape[1] == len(context_token))
        embeddings.append(embedding[0])
    return embeddings, tokenized_context, tokenization

In [11]:
def generate_binary_labels(answers, tokenized_context, tokenizer):
    binary_labels = []
    for k, answer in enumerate(answers):
        answer_texts = answer['text']
        context = tokenized_context[k]
        binary_label = np.zeros(len(context))
        for text in answer_texts:
            tokenization = tokenizer(text, return_tensors="pt")
            answer_token = tokenization['input_ids'][0].detach().numpy()[1:-1]
            answer_length = len(answer_token)
            
            for i in range(len(context) - answer_length + 1):
                if np.all(context[i:i+answer_length] == answer_token):
                    binary_label[i:i+answer_length] = 1
        binary_labels.append(binary_label)

    return binary_labels

In [12]:
def extract_answer_token_and_nonanswer_token(embeddings, binary_labels):
    single_embeddings = []
    single_labels = []
    for i, embedding in enumerate(embeddings):
        for k, token in enumerate(embedding):
            if binary_labels[i][k] == 1:
                single_embeddings.append(token)
                single_labels.append(1)
            else:
                if np.random.randint(0, 100, 1)[0] == 0:
                    single_embeddings.append(token)
                    single_labels.append(0)
      
    return single_embeddings, single_labels

In [13]:
#Extract text-answers from training data
train_dataset = datasets['train']
dev_dataset = datasets['validation']

In [14]:
#数据集过大，取前2000个样本训练
partition = 2000
train_context = train_dataset['context'][:partition]
train_answers = train_dataset['answers'][:partition]
dev_context = dev_dataset['context'][:partition//10]
dev_answers = dev_dataset['answers'][:partition//10]

In [15]:
train_x, train_y = extract_and_tokenize(train_context, train_answers, tokenizer1, encoder1)
dev_x, dev_y = extract_and_tokenize(dev_context, dev_answers, tokenizer1, encoder1)

In [17]:
#训练集中有10000个非答案短语和7700个答案短语
Counter(train_y)

Counter({0: 10107, 1: 7708})

In [19]:
#模型超参
config = {
    'batch_size': 512,
    'dropout_rate': 0.5,
    'learning_rate': 1e-1,
    'epochs': 10
}

In [21]:
#构建dataloader
train_x = torch.Tensor(train_x) 
dev_x = torch.Tensor(dev_x) 
train_y = torch.Tensor(train_y)
dev_y = torch.Tensor(dev_y) 
train_data = TensorDataset(train_x,train_y)
dev_data = TensorDataset(dev_x,dev_y)
train_dataloader = DataLoader(train_data, batch_size = config['batch_size'], 
                              shuffle = True, num_workers = 2) 
dev_dataloader = DataLoader(dev_data, batch_size = config['batch_size'], 
                              shuffle = True, num_workers = 2) 

In [22]:
for x, y in train_dataloader:
    print(x.shape)
    print(y.shape)
    break

torch.Size([512, 128])
torch.Size([512])


# Neural Answer Selector

In [24]:
#普通前馈网络结构
class Network(nn.Module):

    def __init__(self, dropout):

        super(Network, self).__init__()

        input_size = 128
        output_size = 1

        self.model = nn.Sequential(
            #layer 1
            nn.Linear(input_size, input_size * 2),
            nn.BatchNorm1d(input_size * 2),
            nn.LeakyReLU(0.2),
            

            #layer 2
            nn.Dropout(dropout),
            nn.Linear(input_size * 2, input_size * 4),
            nn.BatchNorm1d(input_size * 4),
            nn.LeakyReLU(0.2),

            #layer 3
            nn.Dropout(dropout),
            nn.Linear(input_size * 4, input_size * 8),
            nn.BatchNorm1d(input_size * 8),
            nn.LeakyReLU(0.2),


            #layer 4
            nn.Dropout(dropout),
            nn.Linear(input_size * 8, input_size * 4),
            nn.BatchNorm1d(input_size * 4),
            nn.LeakyReLU(0.2),


            #layer 5
            nn.Dropout(dropout),
            nn.Linear(input_size * 4, input_size * 2),
            nn.BatchNorm1d(input_size * 2),
            nn.LeakyReLU(0.2),

            #layer 6
            nn.Dropout(dropout),
            nn.Linear(input_size * 2, input_size), 
            nn.BatchNorm1d(input_size),
            nn.LeakyReLU(0.2),

            #layer 7
            nn.Dropout(dropout),
            nn.Linear(input_size, input_size // 2), 
            nn.BatchNorm1d(input_size // 2),
            nn.LeakyReLU(0.21),
            
            #layer 8
            nn.Dropout(dropout),
            nn.Linear(input_size // 2, input_size // 4),
            nn.BatchNorm1d(input_size // 4),   
            nn.LeakyReLU(0.2),

            #layer 9
            nn.Dropout(dropout),
            nn.Linear(input_size // 4, input_size // 8), 
            nn.BatchNorm1d(input_size // 8),  
            nn.LeakyReLU(0.2),

            #layer 10
            nn.Linear(input_size // 8, output_size),  
            nn.Sigmoid(),
        )      

    def forward(self, x):
        out = self.model(x)

        return out

In [25]:
model = Network(dropout = config['dropout_rate']).to(device)
token, label = next(iter(train_dataloader))
summary(model, token.to(device))

                        Kernel Shape Output Shape    Params Mult-Adds
Layer                                                                
0_model.Linear_0          [128, 256]   [512, 256]   33.024k   32.768k
1_model.BatchNorm1d_1          [256]   [512, 256]     512.0     256.0
2_model.LeakyReLU_2                -   [512, 256]         -         -
3_model.Dropout_3                  -   [512, 256]         -         -
4_model.Linear_4          [256, 512]   [512, 512]  131.584k  131.072k
5_model.BatchNorm1d_5          [512]   [512, 512]    1.024k     512.0
6_model.LeakyReLU_6                -   [512, 512]         -         -
7_model.Dropout_7                  -   [512, 512]         -         -
8_model.Linear_8         [512, 1024]  [512, 1024]  525.312k  524.288k
9_model.BatchNorm1d_9         [1024]  [512, 1024]    2.048k    1.024k
10_model.LeakyReLU_10              -  [512, 1024]         -         -
11_model.Dropout_11                -  [512, 1024]         -         -
12_model.Linear_12  

/usr/local/lib/python3.9/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_model.Linear_0,"[128, 256]","[512, 256]",33024.0,32768.0
1_model.BatchNorm1d_1,[256],"[512, 256]",512.0,256.0
2_model.LeakyReLU_2,-,"[512, 256]",NaN,NaN
3_model.Dropout_3,-,"[512, 256]",NaN,NaN
4_model.Linear_4,"[256, 512]","[512, 512]",131584.0,131072.0
5_model.BatchNorm1d_5,[512],"[512, 512]",1024.0,512.0
6_model.LeakyReLU_6,-,"[512, 512]",NaN,NaN
7_model.Dropout_7,-,"[512, 512]",NaN,NaN
8_model.Linear_8,"[512, 1024]","[512, 1024]",525312.0,524288.0


In [26]:
criterion = nn.BCELoss() #Defining Loss function 
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'max', factor = 0.7, patience = 5, verbose = True, min_lr = 1e-3)

In [27]:
torch.cuda.empty_cache()
gc.collect()

8

In [28]:
def train(model, optimizer, criterion, dataloader):

    model.train()
    train_loss = 0.0 #Monitoring Loss
    label_true_list = []
    label_pred_list = []
    
    for iter, (tokens, labels) in enumerate(tqdm(dataloader)):

        ### Move Data to Device (Ideally GPU)
        tokens = tokens.to(device)
        labels = labels.to(device)

        ### Forward Propagation
        predictions = model(tokens)

        ### Loss Calculation
        loss = criterion(predictions.reshape(-1), labels)

        train_loss += loss.item()

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Backward Propagation
        loss.backward()

        ### Gradient Descent
        optimizer.step()

        predictions = predictions.detach().numpy().reshape(-1)
        predictions[predictions >= 0.5] = 1
        predictions[predictions < 0.5] = 0

        ### Store Pred and True Labels
        label_pred_list.extend(list(predictions))
        label_true_list.extend(labels.detach().tolist())

        del tokens, labels, predictions
        torch.cuda.empty_cache()


    train_loss /= len(dataloader)
    accuracy = accuracy_score(label_true_list, label_pred_list) 
    return train_loss, accuracy * 100

In [29]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode

    label_true_list = []
    label_pred_list = []
    total_loss = 0.0

    for i, data in enumerate(tqdm(dataloader)):

        tokens, labels = data
        ### Move data to device (ideally GPU)
        tokens, labels = tokens.to(device), labels.to(device) 

        with torch.inference_mode(): # makes sure that there are no gradients computed as we are not training the model now
            ### Forward Propagation
            predictions = model(tokens)
            loss = criterion(predictions.reshape(-1), labels)
            total_loss += float(loss.item())
      
        predictions = predictions.cpu().numpy().reshape(-1)
        predictions[predictions >= 0.5] = 1
        predictions[predictions < 0.5] = 0
        
        
        ### Store Pred and True Labels
        label_pred_list.extend(list(predictions))
        label_true_list.extend(labels.cpu().tolist())
    
        del tokens, labels, predictions
        torch.cuda.empty_cache()

    ### Calculate Accuracy
    accuracy = accuracy_score(label_true_list, label_pred_list) 
    total_loss = float(total_loss / len(dataloader))
    return total_loss, accuracy * 100

# Experiment: Training

In [30]:
# #模型已经训练好了
# # Iterate over number of epochs to train and evaluate your model
# torch.cuda.empty_cache()

# best_accuracy = 0

# for epoch in range(config['epochs']):
#     print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

#     train_loss, train_accuracy = train(model, optimizer, criterion, train_dataloader)
#     val_loss, val_accuracy = eval(model, dev_dataloader)

#     scheduler.step(val_loss)

#     print("Train Loss: {:.4f}".format(train_loss))
#     print("Training Accuracy: {:.2f}%".format(train_accuracy))
#     print("Validation Loss: {:.4f}".format(val_loss))
#     print("Validation Accuracy: {:.2f}%".format(val_accuracy))


#     if val_accuracy > best_accuracy:
#         best_accuracy = val_accuracy
#         print("Saving model")
#         torch.save({'model_state_dict':model.state_dict(),
#                     'optimizer_state_dict':optimizer.state_dict(),
#                     'val_accuracy': val_accuracy, 
#                     'epoch': epoch}, '/content/drive/MyDrive/11611/Project/answer_extractor.pth')


# Inference

In [31]:
#Load trained model
#模型已经训练好了， 载入模型
PATH = '/content/drive/MyDrive/11611/Project/answer_extractor.pth'
QE_model = Network(dropout = config['dropout_rate']).to(device)
QE_model.load_state_dict(torch.load(PATH)['model_state_dict'])
QE_model.eval()

Network(
  (model): Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=256, out_features=512, bias=True)
    (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): LeakyReLU(negative_slope=0.2)
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=512, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2)
    (11): Dropout(p=0.5, inplace=False)
    (12): Linear(in_features=1024, out_features=512, bias=True)
    (13): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): LeakyReLU(negative_slope=0.2)
    (15): Dropout(p=0.5, inplace=False)
    (16): Linear(in_features=512, out_f

In [32]:
#以下系列函数都是用来把模型预测属于答案的embeddings对应回原文相应的短语，返回一个包含答案短语字符串的[]
def predict_answer(model, context, tokenizer, encoder):
    words_list, words_ids_list, words_mapping = context_to_words_inputids(context, tokenizer)
    embeddings, _, tokenization = tokenize_and_encode_context([context], tokenizer, encoder)
    predictions = model_predict(model, embeddings)
    input_ids = tokenization['input_ids'][0].detach().numpy().tolist()
    answer_ids_list = get_answer_ids(input_ids, predictions)
    answers = map_answer_ids_to_words(answer_ids_list, words_ids_list, words_mapping, words_list)
    return sorted(set(answers), key=answers.index)

In [33]:
def map_answer_ids_to_words(answer_ids_list, words_ids_list, words_mapping, words_list):
    text_answer_list = []
    for ans in answer_ids_list:
        ans_length = len(ans)
        for i in range(len(words_ids_list) - ans_length + 1):
            if words_ids_list[i:i+len(ans)] == ans:
                corresponding_words = words_mapping[i:i+len(ans)]
                corresponding_words = sorted(set(corresponding_words), key=corresponding_words.index)
                text_ans = ""
                for w in corresponding_words:
                    text_ans += ' ' + words_list[w]
                text_answer_list.append(text_ans[1:])
    return text_answer_list

In [34]:
def model_predict(model, embeddings):
    embeddings = torch.tensor(embeddings[0])
    predictions = model(embeddings).flatten()
    predictions[predictions >= 0.5] = 1
    predictions[predictions < 0.5] = 0
    predictions = predictions.detach().numpy().tolist()
    return predictions

In [35]:
def get_answer_ids(input_ids, predictions):
    assert(len(input_ids) == len(predictions))
    all_answers_ids = []
    
    i = 0
    while i < len(predictions) - 1:
        if predictions[i] == 1:
            ans_span = [input_ids[i]]
            j = i + 1
            while j < len(predictions):
                if predictions[j] == 1:
                    ans_span.append(input_ids[j])
                    j += 1
                else:
                    all_answers_ids.append(ans_span)
                    i = j
                    break
        else:
            i += 1
            
    return all_answers_ids

In [38]:
def context_to_words_inputids(context,  tokenizer):
    word_ids_list = []
    word_mapping = []
    words_list = context_to_words(context)
    for k, word in enumerate(words_list):
        word_id = tokenizer(word)['input_ids'][1:-1]
        word_ids_list.extend(word_id)
        word_mapping.extend([k for i in range(len(word_id))])
    return words_list, word_ids_list, word_mapping

In [37]:
def context_to_words(context):
    return re.sub("[^\w]", " ",  context).split()

In [39]:
#since in the dataset a lot of context is same, we remove duplicates
#测试：去除测试集中的重复文章（测试集都是一个文章对好几个答案）
dev_context_no_dup = list(set(dev_context))

In [40]:
#对每篇测试文章都生生成对应的若干答案
pred_answers = []
for cont in tqdm(dev_context_no_dup):
    pred_answer = predict_answer(QE_model, cont, tokenizer1, encoder1)
    pred_answers.append(pred_answer)

100%|██████████| 38/38 [00:02<00:00, 15.90it/s]


In [ ]:
# with open("/content/drive/MyDrive/11611/Project/context.txt", "w") as f:
#     for cont in dev_context_no_dup:
#         f.write(cont)
#         f.write('\n')

In [ ]:
# with open("/content/drive/MyDrive/11611/Project/potential_answers.txt", "w") as f:
#     for answers in pred_answers:
#         n = len(answers)
#         for i, ans in enumerate(answers):
#             f.write(ans)
#             if i == n - 1:
#                 continue
#             f.write(',')
#         f.write('\n')

#Generate Questions

In [43]:
#载入问题生成模型的预训练模型和配套的的tokenizer
from transformers import AutoModelWithLMHead, AutoTokenizer

In [44]:
model_name = "mrm8488/t5-base-finetuned-question-generation-ap"
tokenizer2 = AutoTokenizer.from_pretrained(model_name, use_fast=False)
QG_model = AutoModelWithLMHead.from_pretrained(model_name)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1295: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [48]:
#QG的主函数，对于一篇文章context，生成N个问题，若能够生成的问题大于N个，则返回前N个问题
def generate_questions(context, N = 10):
    answers = predict_answer(QE_model, context, tokenizer1, encoder1)
    questions = []
    for i in range(len(answers)):
        this_answer = answers[i]
        pred_quesiton = get_question(this_answer, context)
        questions.append(pred_quesiton)
    if len(questions) > N:
        return questions[:N]
    else:
        return questions

In [49]:
def get_question(answer, context, max_length=256):
  input_text = "answer: %s  context: %s </s>" % (answer, context)
  features = tokenizer2([input_text], return_tensors='pt')

  output = QG_model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)
  question = tokenizer2.decode(output[0], skip_special_tokens=True)
  return question

# EXAMPLE

In [50]:
context = dev_context_no_dup[0]
print(context)

Normans came into Scotland, building castles and founding noble families who would provide some future kings, such as Robert the Bruce, as well as founding a considerable number of the Scottish clans. King David I of Scotland, whose elder brother Alexander I had married Sybilla of Normandy, was instrumental in introducing Normans and Norman culture to Scotland, part of the process some scholars call the "Davidian Revolution". Having spent time at the court of Henry I of England (married to David's sister Maud of Scotland), and needing them to wrestle the kingdom from his half-brother Máel Coluim mac Alaxandair, David had to reward many with lands. The process was continued under David's successors, most intensely of all under William the Lion. The Norman-derived feudal system was applied in varying degrees to most of Scotland. Scottish families of the names Bruce, Gray, Ramsay, Fraser, Ogilvie, Montgomery, Sinclair, Pollock, Burnard, Douglas and Gordon to name but a few, and including 

In [51]:
generated_questions = generate_questions(context, N = 10)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


In [52]:
for q in generated_questions:
    print(q)

question: Who built castles and founded noble families?
question: Where did Normans come to?
question: What did the Normans do when they came into Scotland?
question: What did Normans do when they came into Scotland?
question: What did the Normans provide to Scotland?
question: What was Robert the Bruce's name?
question: How many Scottish clans were founded by Normans?
question: What was King David I instrumental in doing?
question: How long did Henry I spend at the court of Normandy?
question: Where did David spend time?
